### Rendering

### Interactively

In [1]:
from zxopt.visualization import Window

window = Window()
window.main_loop()


KeyboardInterrupt: 

### OpenQASM

In [ ]:
from zxopt.openqasm import OpenQasmParser

parser = OpenQasmParser()
circuit = parser.load_file("./circuits/bell_swap.qasm")

### ZX-diagram


In [ ]:
from zxopt.data_structures.diagram import Diagram
from zxopt.visualization import DiagramRenderer
import math

diagram = Diagram()
in1 = diagram.add_boundary("in", 0)
in2 = diagram.add_boundary("in", 1)
out1 = diagram.add_boundary("out", 0)
out2 = diagram.add_boundary("out", 1)

s1_1 = diagram.add_spider(0.0, "green", 0)
s1_2 = diagram.add_spider(0.0, "red", 1)
s2_1 = diagram.add_spider(math.pi, "red", 0)

diagram.add_wire(in1, s1_1)
diagram.add_wire(in2, s1_2)
diagram.add_wire(s1_1, s1_2)
diagram.add_wire(s1_1, s2_1)
diagram.add_wire(s2_1, out1)
diagram.add_wire(s1_2, out2)

# diagram.render()
diagram_renderer = DiagramRenderer(diagram)
diagram_renderer.render_image()

### Circuit translation

In [ ]:
from zxopt.translation import CircuitTranslator
from zxopt.visualization import Window
from zxopt.openqasm import OpenQasmParser
from zxopt.visualization import DiagramRenderer

circuit = OpenQasmParser().load_file("./circuits/test/simple_translation_test.qasm")
diagram = CircuitTranslator(circuit).translate()

renderer = DiagramRenderer(diagram)
# renderer = CircuitRenderer(circuit)

window = Window(renderer)
window.main_loop()

### Validation

#### Circuit

In [ ]:
from zxopt.data_structures.circuit import Circuit, GateComponent, HadamardGateType, PauliXGateType
from zxopt.data_structures.circuit.register.quantum_register import QuantumRegister
from zxopt.visualization import CircuitRenderer
from zxopt.validation import CircuitUnitaryExtractor, DiagramLinearExtractor
from zxopt.visualization import DiagramRenderer
from zxopt.openqasm import OpenQasmParser

# circuit = Circuit()
# qreg = QuantumRegister(2, "q")
# circuit.add_register(qreg)
# h = GateComponent(qreg[0], HadamardGateType())
# cx = GateComponent(qreg[1], PauliXGateType(), control_bits={qreg[0]})
# circuit.add_component(h)
# circuit.add_component(cx)
circuit = OpenQasmParser().load_file("circuits/test/bell_state_test_circuit.qasm")

renderer = CircuitRenderer(circuit)
renderer.render_image()

circuitValidator = CircuitUnitaryExtractor(circuit)
circuitExtractedMatrix = circuitValidator.extract_matrix()


#### Validation tensor contraction

In [ ]:
import tensornetwork as tn
import math
import numpy as np

# represents same circuit as above
H = math.sqrt(2) * np.array([[1,1],[1,-1]])
Z3 = np.array([[[1,0],[0,0]],[[0,0],[0,1]]]) # alpha = 0
A = H
B = H
C = Z3
D = H
E = Z3
F = H

linearMap = tn.ncon([A,B,C,D,E,F], [(-3, 1), (-4, 2), (1, -1, 3), (3,4), (2,4,5), (5,-2)])
np.round(linearMap.reshape(4,4), decimals=5) # same as above scaled

# dimension 1 and 2 are input, 3 and 4 are output (how to reshape correctly)
# just swap dimension ids as input to ncon?


#### Diagram

In [ ]:
from zxopt.translation import CircuitTranslator
from zxopt.validation import DiagramLinearExtractor
from zxopt.visualization import DiagramRenderer
from zxopt.visualization import Window

diagram = CircuitTranslator(circuit).translate()

diagram_renderer = DiagramRenderer(diagram, disable_alignment=False)
diagram_renderer.render_image()

window = Window(diagram_renderer)
window.main_loop()

diagramValidator = DiagramLinearExtractor(diagram)
diagramExtractedMatrix = np.round(diagramValidator.extract_matrix(), decimals=5)

#### Comparison

In [ ]:
from zxopt.validation import validate_operation_equality
import math

print(circuitExtractedMatrix)
print(diagramExtractedMatrix * np.exp(1j * math.pi / 4.0))
print("Equality: ", validate_operation_equality(circuitExtractedMatrix, diagramExtractedMatrix))